In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [2]:
# Step 1: Prepare Your Dataset
train_data_dir = 'C:\\Users\\ABHISHEK\\Downloads\\final dataset - Copy\\train'
validation_data_dir = 'C:\\Users\\ABHISHEK\\Downloads\\final dataset - Copy\\test'
img_width, img_height = 224, 224
batch_size = 32

In [3]:
# Step 2: Load and Preprocess Your Dataset
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 3900 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


In [4]:
# Step 3: Create a MobileNet Model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the MobileNet layers
for layer in base_model.layers:
    layer.trainable = False

# Step 4: Add Additional Layers
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(6, activation='softmax'))

# Step 5: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train the Model
epochs = 5
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size)


17225924/17225924 [==============================] - 3s 0us/step
Epoch 1/5
121/121 [==============================] - 189s 2s/step - loss: 0.8903 - accuracy: 0.6691 - val_loss: 1.4553 - val_accuracy: 0.4983
Epoch 2/5
121/121 [==============================] - 145s 1s/step - loss: 0.6284 - accuracy: 0.7668 - val_loss: 1.0144 - val_accuracy: 0.6528
Epoch 3/5
121/121 [==============================] - 141s 1s/step - loss: 0.5342 - accuracy: 0.8082 - val_loss: 1.0090 - val_accuracy: 0.6319
Epoch 4/5
121/121 [==============================] - 132s 1s/step - loss: 0.4714 - accuracy: 0.8307 - val_loss: 1.2036 - val_accuracy: 0.6285
Epoch 5/5
121/121 [==============================] - 230s 2s/step - loss: 0.4319 - accuracy: 0.8444 - val_loss: 0.9800 - val_accuracy: 0.6771


In [5]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 3,492,806
Trainable params: 263,942
Non-trainable params: 3,228,864
_________________________________________________________________


In [6]:
accuracy = model.evaluate(validation_generator)
print("Validation Accuracy:", accuracy)
accuracy = model.evaluate(train_generator)
print("Training Accuracy:", accuracy)

19/19 [==============================] - 21s 1s/step - loss: 0.9952 - accuracy: 0.6700
Validation Accuracy: [0.9951652884483337, 0.6700000166893005]
122/122 [==============================] - 227s 2s/step - loss: 0.3764 - accuracy: 0.8664
Training Accuracy: [0.3763522207736969, 0.8664102554321289]
